<a href="https://colab.research.google.com/github/harshiniKumar/GPT-Neo_SQUAD/blob/main/GPT_Neo_SQUAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/EleutherAI/gpt-neo.git

Cloning into 'gpt-neo'...
remote: Enumerating objects: 3866, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 3866 (delta 234), reused 276 (delta 160), pack-reused 3468
Receiving objects: 100% (3866/3866), 1.45 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (2234/2234), done.


In [ ]:
%cd gpt-neo
!pip install -r requirements.txt

/content/gpt-neo
     |████████████████████████████████| 361 kB 34.0 MB/s 
     |████████████████████████████████| 14.5 MB 47.5 MB/s 
     |████████████████████████████████| 106 kB 58.2 MB/s 
     |████████████████████████████████| 454.4 MB 9.8 kB/s 
     |████████████████████████████████| 3.4 MB 31.1 MB/s 
     |████████████████████████████████| 2.9 MB 45.4 MB/s 
     |████████████████████████████████| 1.5 MB 35.7 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 4.0 MB 35.5 MB/s 
     |████████████████████████████████| 1.2 MB 59.6 MB/s 
     |████████████████████████████████| 462 kB 49.0 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
     |████████████████████████████████| 2.9 MB 20.6 MB/s 
     |████████████████████████████████| 216 kB 52.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.8 MB/s 
     |████████████████████████████████| 132 kB 73.5 MB/s 
     |████████████████████████████████| 99 kB 10.2 MB/s

## Preprocessing

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
file_path = "/content/dataset/merged.json"
with open(file_path, "rb") as f:
  file_ = json.load(f)
data = file_["data"][0]["paragraphs"]
context_array = []
question_array = []
answer_array = []
resumes = []
print(len(data))
for resume in data[:3000]:
  qas = resume["qas"]
  resumes.append(resume["context"][:2048])
  for qa in qas:
    question = qa["question"]
    answer = qa["answers"][0]["text"]
    question_array.append(question)
    context_array.append(resume["context"])
    answer_array.append(answer)

resumes = resumes[:4000]
qa_df = pd.DataFrame({
   'context' : context_array,
   'question' : question_array,
   'answer' : answer_array, 
})
print(qa_df)

66
                                               context  ...                          answer
0    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...               in the late 1990s
1    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...             singing and dancing
2    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                            2003
3    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                  Houston, Texas
4    Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                      late 1990s
..                                                 ...  ...                             ...
748  In December, Beyoncé along with a variety of o...  ...                        Catapult
749  In December, Beyoncé along with a variety of o...  ...                   Demand A Plan
750  In December, Beyoncé along with a variety of o...  ...    Sandy Hook Elementary School
751  In December, Beyoncé along with a variety of o...  ...                  

In [ ]:
np.savetxt(r'/content/dataset/merged.txt',qa_df.values,fmt='%s',delimiter='\t')

##Train Tokenizer

In [ ]:
!python data/tokenizer.py
  --base_dir  "" \
  --output_dir "" \
  --file_type "" \
  --vocab_size 50257

##Conversion to tfrecords

**input_dir**: Defines the folder where you data is located. The script will encode all files present in this folder.

**name**: Name of output files will be name_i.tfrecords where i is the number of the files.

**output_dir**: Where to save the tf records to

**use_gpt2_tokenizer** Whether to use the pretrained HuggingFace GPT2



In [ ]:
import os
dataset_path = "/content/dataset/"
dataset_name = "merged"
out_path = "/content/dataset/"
tokenizer_path = ""

In [ ]:
# Tokenize Data
!python data/create_tfrecords.py --input_dir $dataset_path --name $dataset_name --files_per 1000 --output_dir $out_path --write_dataset_config --processes 1


2021-12-08 05:01:12.681910: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Writing TFRecord Files to /content/dataset/. Parsed 0 input files. files_written : 0it [00:01, ?it/s]
{'discarded': 0, 'processed': 1, 'successful': 1}


##Model Configuration

In [ ]:
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/"

--2021-12-08 05:01:40--  https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/
Resolving the-eye.eu (the-eye.eu)... 162.213.130.244
Connecting to the-eye.eu (the-eye.eu)|162.213.130.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’

the-eye.eu/public/A     [ <=>                ]  31.91K  --.-KB/s    in 0s      

2021-12-08 05:01:40 (183 MB/s) - ‘the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp’ saved [32672]

Loading robots.txt; please ignore errors.
--2021-12-08 05:01:42--  https://the-eye.eu/robots.txt
Reusing existing connection to the-eye.eu:443.
HTTP request sent, awaiting response... 404 Not Found
2021-12-08 05:01:42 ERROR 404: Not Found.

Removing the-eye.eu/public/AI/gptneo-release/GPT3_XL/index.html.tmp since it should be rejected.

--2021-12-08 05:01:44--  https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/?sort=namedirfirst&order=asc
Reusing e

In [ ]:
pretrained_model = 'GPT3_XL' 
path_to_local_weights = f"/content/gpt-neo/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"
dataset = None

In [19]:
import json
from pprint import pprint

path_to_model = "/content/gpt-neo/the-eye.eu/public/AI/gptneo-release/GPT3_XL/" 
batch_size = 8 
dset = "GPT_XL.json" 
mesh_shape = "x:1,y:1" 
train_steps = 1000 
steps_per_checkpoint = 500
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: /content/gpt-neo/the-eye.eu/public/AI/gptneo-release/GPT3_XL/

{'activation_function': 'gelu',
 'attention_types': [[['global', 'local'], 12]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,memory_length:y,embd:y',
 'lr': 0.0002,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:128,y:2',
 'model_path': 'gs://neo-d/models/GPT3_XL_Pile',
 'n_ctx': 2048,
 'n_embd': 2048,
 'n_head': 16,
 'n_layer': 24,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'precision': 'bfloat16',
 'predict_batch_size': 128,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_steps': 3000,
 'weight_decay

In [20]:
%%writefile configs/dataset_configs/GPT_XL.json

{
  "path": "/content/dataset/merged_0_76.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Overwriting configs/dataset_configs/GPT_XL.json


In [21]:
pwd

'/content/gpt-neo'

##Run Training

Using Auto Layout and Mesh Shape

In [22]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --auto_layout_and_mesh_shape 

2021-12-08 05:27:18.591511: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Traceback (most recent call last):
  File "main.py", line 257, in <module>
    main(args)
  File "main.py", line 56, in main
    params = fetch_model_params(args.model)
  File "/content/gpt-neo/configs.py", line 29, in fetch_model_params
    assert dataset_id in DATASETS, f"Dataset '{dataset_id}' was not found under dataset_configs/ folder. Please follow the example.json in that folder."
AssertionError: Dataset 'GPT_XL.json' was not found under dataset_configs/ folder. Please follow the example.json in that folder.


Model Training

In [23]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

2021-12-08 05:28:07.209400: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Traceback (most recent call last):
  File "main.py", line 257, in <module>
    main(args)
  File "main.py", line 56, in main
    params = fetch_model_params(args.model)
  File "/content/gpt-neo/configs.py", line 29, in fetch_model_params
    assert dataset_id in DATASETS, f"Dataset '{dataset_id}' was not found under dataset_configs/ folder. Please follow the example.json in that folder."
AssertionError: Dataset 'GPT_XL.json' was not found under dataset_configs/ folder. Please follow the example.json in that folder.
